### Postcode Analysis

In [1]:
from pyspark.sql import SparkSession, functions as F

sp = (
    SparkSession.builder.appName("Data_Explorer")
    #.config("spark.sql.repl.eagerEval.enabled", True) 
    #.config("spark.sql.parquet.cacheMetadata", "true")
    #.config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/10/06 12:05:05 WARN Utils: Your hostname, J-L resolves to a loopback address: 127.0.1.1; using 172.19.38.60 instead (on interface eth0)
22/10/06 12:05:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/06 12:05:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/06 12:05:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/06 12:05:08 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


# Postcode Transaction Analysis
The objective of this section is to verify if a customer's details have been recorded correctly

In [2]:
customers = sp.read.option("inferSchema", True).parquet("../data/curated/consumer_details/")
fraud = sp.read.option("inferSchema", True).parquet("../data/curated/customer_fraud")

customers.show(1)
fraud.show(1)

+---------------+--------------------+-----+--------+------+-------+
|           name|             address|state|postcode|gender|user_id|
+---------------+--------------------+-----+--------+------+-------+
|Courtney Mendez|802 Knapp Harbors...|  QLD|    9013|Female|  71144|
+---------------+--------------------+-----+--------+------+-------+
only showing top 1 row

+-------+-------------------+-----------------+
|user_id|     order_datetime|fraud_probability|
+-------+-------------------+-----------------+
|   6228|2021-12-19 00:00:00|         97.62981|
+-------+-------------------+-----------------+
only showing top 1 row



In [4]:
from pyspark.sql.types import DateType

fraud = fraud.withColumn("order_datetime", F.col("order_datetime").cast(DateType()))
c_fraud = fraud.join(customers.select("user_id", "postcode"), on="user_id")

c_fraud.show(2)

+-------+--------------+-----------------+--------+
|user_id|order_datetime|fraud_probability|postcode|
+-------+--------------+-----------------+--------+
|      1|    2022-02-20|         9.805431|    6935|
|      2|    2021-08-29|         9.599514|    2782|
+-------+--------------+-----------------+--------+
only showing top 2 rows



In [6]:
import folium
import pandas as pd
import geopandas as gpd

In [7]:
zones = pd.read_csv("../data/tables/australian_postcodes.csv")
sf = gpd.read_file("../data/tables/SA2_2021_AUST_GDA2020.shp")

In [9]:
zones.head(2)

,id,postcode,locality,state,long,lat,dc,type,status,sa3,...,MMM_2015,MMM_2019,ced,altitude,chargezone,phn_code,phn_name,lgaregion,electorate,electoraterating
0,230,200,ANU,ACT,149.1190,-35.2777,NaN,NaN,NaN,NaN,...,1.0,1.0,NaN,NaN,N2,NaN,NaN,NaN,Durack,NaN
1,21820,200,Australian National University,ACT,149.1189,-35.2777,NaN,NaN,Added 19-Jan-2020,NaN,...,1.0,1.0,NaN,NaN,N2,NaN,NaN,NaN,Durack,NaN


In [10]:
sf.head(2)

,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
0,101021007,Braidwood,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,3418.3525,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.58424 -35.44426, 149.58444 -35.4..."
1,101021008,Karabar,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,6.9825,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.21899 -35.36738, 149.21800 -35.3..."


In [27]:
from codecs import ascii_encode

dataset.groupBy(['electoraterating','state']).agg(F.avg('dollar_value').alias('dollar_avg'), F.avg('Proportion with salary or wages').alias('AVG_job_pot')).sort('dollar_avg', ascending=False)

electoraterating,state,dollar_avg,AVG_job_pot
Outer Metropolitan,WA,162.10137457495566,1.0
Inner Metropolitan,QLD,161.95596260030928,1.0
Rural,ACT,161.37831281349753,1.0
Outer Metropolitan,TAS,161.1370434353406,1.0
Inner Metropolitan,TAS,161.03532053220445,1.0
Rural,NT,160.41877305512617,1.0
Inner Metropolitan,ACT,160.1090136904166,1.0
Inner Metropolitan,SA,159.87970617166403,1.0
Rural,QLD,159.46443485014788,1.0
Rural,TAS,159.31208605207823,1.0
